In [ ]:
from ipywidgets import widgets
from IPython.display import display
# Create a text input widget
input_name = widgets.Text(placeholder='Enter your name')
# Display the text input widget
display(input_name)
NAME = "-"
# Function to handle text input change
def handle_text_change(change):
    global NAME
    NAME = change["new"]
    print("Entered Name:", NAME)
    #   ENTER '$' TO STOP TAKING INPUT
    if '$' in NAME:
        input_name.unobserve(handle_text_change, names='value')
        NAME = NAME[:-1]
        print(NAME)

# Register the handle_text_change function to be called when the text input changes
input_name.observe(handle_text_change, names='value')

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pyttsx3
import threading
import math

# WIDTH = 960 #640
# HEIGHT = 540 #480
WIDTH = 640
HEIGHT = 480

#contants
margin_left = int(WIDTH*0.25)	#margin left (tools)
# margin_left = 150	#margin left (tools)
max_x, max_y = 250 + margin_left, 50	#from 0,0 dist of tools in x-axis(margin_left + width) and y-axis(height)
curr_tool = "select tool"
time_init = True
rad = 40
var_inits = False
thick = 4
prevx, prevy = 0,0


MAX_SLEEP_TIME = 100
sleep_timer = MAX_SLEEP_TIME

# NAME = "Harsha"
print("Name :",NAME)

def getX(x):
	return x*WIDTH
def getY(y):
	return y*HEIGHT

#GET SIGN'S
def getSign(finger_landmarks):
	fingers = []
	tipIds = [4, 8, 12, 16, 20]
	#Thumb finger	#------NOT WORKING WELL --------
	x1,x2 = getX(finger_landmarks[tipIds[0]].x) , getX(finger_landmarks[tipIds[0] - 1].x)	
	y1,y2 = getX(finger_landmarks[tipIds[0]].y) , getX(finger_landmarks[tipIds[0] - 1].y)
	x3,y3 = getX(finger_landmarks[tipIds[1]-2].x) , getX(finger_landmarks[tipIds[1]-2].y)
	if (x2 > x1 or y2 > y1) and math.sqrt(math.pow(x1-x2, 2) + math.pow(y1-y2, 2)) > 40 and math.sqrt(math.pow(x3-x1, 2) + math.pow(y3-y1, 2)) > 50:
			fingers.append(1)
	else:
		fingers.append(0)

	# 4 Fingers
	for id in range(1, 5):
		x1,x2 = getX(finger_landmarks[tipIds[id]].x) , getX(finger_landmarks[tipIds[id] - 2].x)
		y1,y2 = getX(finger_landmarks[tipIds[id]].y) , getX(finger_landmarks[tipIds[id] - 2].y)
		if y2 > y1 and math.sqrt(math.pow(x1-x2, 2) + math.pow(y1-y2, 2)) > 50:
			fingers.append(1)
		else:
			fingers.append(0)
	# print(fingers)

	if fingers == [1, 1, 1, 1, 1]:
		return {"text": f"{NAME} says hello", "message": "Hello"}
	elif fingers == [1, 0, 0, 0, 0]:
		return {"text": f"{NAME} mentions the need for water", "message": "Need Water"}
	elif fingers == [0, 0, 0, 0, 1]:
		return {"text": f"{NAME} mentions the need to use the restroom", "message": "Washroom"}
	elif fingers == [0, 0, 1, 1, 1]:
		return {"text": f"{NAME} indicates feeling well", "message": "I'm Good"}
	elif fingers == [0, 0, 0, 0, 0]:
		return {"text": f"{NAME} mentions experiencing discomfort", "message": "In Pain"}
	# elif fingers == [1, 0, 0, 0, 1]:
	# 	return {"text": f"{NAME} indicates the need to make a call", "message": "Need to call"}
	return {}
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

#get tools function
def getTool(x):
	# ----ADD CLEAR ALL----
	print("tool x: ",x)
	if x < 50 + margin_left:
		return "line"

	elif x< 100 + margin_left:
		return "rectangle"

	elif x < 150 + margin_left:
		return"draw"

	elif x<200 + margin_left:
		return "circle"

	else:
		return "erase"

def index_raised(yi, y9):
	if (y9 - yi) > 40:	# difference between middle finger's bottom line and tip > 40
		return True
	return False



hands = mp.solutions.hands
hand_landmark = hands.Hands(min_detection_confidence=0.6, min_tracking_confidence=0.6, max_num_hands=1)
draw = mp.solutions.drawing_utils


# drawing tools
tools = cv2.imread("tools.png")
tools = tools.astype('uint8')

mask = np.ones((HEIGHT, WIDTH))*255	# white mask(total img screen) created with width-640px height-480px
mask = mask.astype('uint8')
'''
tools = np.zeros((max_y+5, max_x+5, 3), dtype="uint8")
cv2.rectangle(tools, (0,0), (max_x, max_y), (0,0,255), 2)
cv2.line(tools, (50,0), (50,50), (0,0,255), 2)
cv2.line(tools, (100,0), (100,50), (0,0,255), 2)
cv2.line(tools, (150,0), (150,50), (0,0,255), 2)
cv2.line(tools, (200,0), (200,50), (0,0,255), 2)
'''

# cap = cv2.VideoCapture(0)	#object of 0 - default camera
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

while False:
	# width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
	# height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
	# print('width, height:', width, height)
	success, frame = cap.read()	#RETURN capture success/ failure , frame
	if(not success):
		continue
	frame = cv2.flip(frame, 1)	#flip horizontally(1) , as webcam images are mirrored

	rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)	# OpenCV reads images in BGR format by default

	op = hand_landmark.process(rgb)
	
	sleep_timer = min(MAX_SLEEP_TIME, sleep_timer + 1)
	# print(op.multi_hand_landmarks)
	if op.multi_hand_landmarks:	# array of all hand land marks x,yz
		# print(len(op.multi_hand_landmarks))
		for i in op.multi_hand_landmarks:	# it contains only one object i = op.multi_hand_landmarks[0]
			# print(i)
			# print(hands.HAND_CONNECTIONS)
			draw.draw_landmarks(frame, i, hands.HAND_CONNECTIONS)	#draw handmarks on the frame screen
			x, y = int(getX(i.landmark[8].x)), int(getY(i.landmark[8].y))	#landmark[8] - index fingers tip
			# index finger is on tools
			if x < max_x and y < max_y and x > margin_left:
				if time_init:	#first time entering into tools region
					ctime = time.time() #set entered time
					time_init = False
				ptime = time.time()	#set current time
				cv2.circle(frame, (x, y), rad, (0, 255,0), 2)	#rad-radius, (b,g,r) , 2-thickness
				rad -= 1	#for pulse on circle radius

				if (ptime - ctime) > 0.8:	# if it is in more than 0.8 sec then select that tool
					curr_tool = getTool(x)
					print("your current tool set to : ", curr_tool)
					time_init = True
					rad = 40

			else:
				time_init = True
				rad = 40

				# HANDLING SINGS
				if sleep_timer == MAX_SLEEP_TIME:
					# curr_tool = "select tool"
					signText = getSign(i.landmark)
					if signText != {}:
						print(signText["text"])
						curr_tool = signText["message"]
						t = threading.Thread(target=speak, args=(signText["text"],))
						t.start()
						sleep_timer = 0


			# free hand draw
			if curr_tool == "draw":
				xi, yi = int(getX(i.landmark[12].x)), int(getY(i.landmark[12].y))
				y9  = int(getY(i.landmark[9].y))	#landmark[9] - middle fingers bottom line 

				if index_raised(yi, y9):
					cv2.line(mask, (prevx, prevy), (x, y), 0, thick)
					prevx, prevy = x, y

				else:
					prevx = x
					prevy = y



			elif curr_tool == "line":
				xi, yi = int(getX(i.landmark[12].x)), int(getY(i.landmark[12].y))
				y9  = int(getY(i.landmark[9].y))	#landmark[9] - middle fingers bottom line 

				if index_raised(yi, y9):
					if not(var_inits):
						xii, yii = x, y
						var_inits = True

					cv2.line(frame, (xii, yii), (x, y), (50,152,255), thick)

				else:
					if var_inits:
						cv2.line(mask, (xii, yii), (x, y), 0, thick)
						var_inits = False

			elif curr_tool == "rectangle":
				xi, yi = int(getX(i.landmark[12].x)), int(getY(i.landmark[12].y))
				y9  = int(getY(i.landmark[9].y))	#landmark[9] - middle fingers bottom line 

				if index_raised(yi, y9):
					if not(var_inits):
						xii, yii = x, y
						var_inits = True

					cv2.rectangle(frame, (xii, yii), (x, y), (0,255,255), thick)

				else:
					if var_inits:
						cv2.rectangle(mask, (xii, yii), (x, y), 0, thick)
						var_inits = False

			elif curr_tool == "circle":
				xi, yi = int(getX(i.landmark[12].x)), int(getY(i.landmark[12].y))
				y9  = int(getY(i.landmark[9].y))	#landmark[9] - middle fingers bottom line 

				if index_raised(yi, y9):
					if not(var_inits):
						xii, yii = x, y
						var_inits = True

					cv2.circle(frame, (xii, yii), int(((xii-x)**2 + (yii-y)**2)**0.5), (255,255,0), thick)

				else:
					if var_inits:
						cv2.circle(mask, (xii, yii), int(((xii-x)**2 + (yii-y)**2)**0.5), 0, thick)
						var_inits = False

			elif curr_tool == "erase":
				xi, yi = int(getX(i.landmark[12].x)), int(getY(i.landmark[12].y))
				y9  = int(getY(i.landmark[9].y))	#landmark[9] - middle fingers bottom line 

				if index_raised(yi, y9):
					cv2.circle(frame, (x, y), 30, (0,0,0), -1)
					cv2.circle(mask, (x, y), 30, 255, -1)	#clearing into white



	op = cv2.bitwise_and(frame, frame, mask=mask)
	frame[:, :, 0] = op[:, :, 0]	#replaces the Blue channel of the frame with the corresponding values from the op image.
	frame[:, :, 1] = op[:, :, 1]	#replaces the Green channel of the frame with the corresponding values from the op image.
	frame[:, :, 2] = op[:, :, 2]	#replaces the Red channel of the frame with the corresponding values from the op image.

	frame[:max_y, margin_left:max_x] = cv2.addWeighted(tools, 0.9, frame[:max_y, margin_left:max_x], 0.2, 0)	#display tools

	cv2.putText(frame, curr_tool, (270+margin_left,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)	#display selected name
	cv2.imshow("Gesture Recognition Application", frame)	#display the screen

	#press Esc to quit
	if cv2.waitKey(1) == 27:
		cv2.destroyAllWindows()
		cap.release()
		break
